<a href="https://colab.research.google.com/github/buuck/CAISO_renewable_energy/blob/main/workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install pydeck geopandas

# 0. Setup

In [ ]:
import pandas as pd # must be imported before threading
import duckdb
import pandas as pd
import geopandas as gpd
from shapely import wkt
import shapely
import geopandas as gpd
import numpy as np
from threading import Thread, current_thread
import random


con = duckdb.connect('db.duckdb')
con.sql("INSTALL spatial; LOAD spatial; INSTALL httpfs; LOAD httpfs;")

## 0.1. Imports

## 0.2. Load data

### 0.2.1 Species ICUN ratings

#### ICUN Red List 5 point scale
5 point scale from most (5) to least (1) threathened species
- Rank 1 = Least Concern, Data Deficient & Not Evaluated.
- Rank 2 = Near Threatened
- Rank 3 = Vulnerable
- Rank 4 = Endangered
- Rank 5 = Extinct in the Wild & Critically Endangered


In [ ]:
# Species table

data = [
    # {'Species': 'Orca', 'Order': 'Cetacea', 'Family': 'Delphinidae', 'Common Name': 'Orca'}, # NOT present
    {'Species': 'Salmon', 'Order': 'Salmoniformes', 'Family': 'Salmonidae', 'CommonName': 'Salmon', "Icun": 3},  # present in dataset
    {'Species': 'Seals', 'Order': 'Carnivora', 'Family': 'Phocidae', 'CommonName': 'Seals', "Icun": 1}, # present in dataset
    {'Species': 'White Spotted Owl', 'Order': 'Strigiformes', 'Family': 'Strigidae', 'CommonName': 'White Spotted Owl', "Icun": 3}, # present in dataset
    {'Species': 'Marbled Murrelet', 'Order': 'Charadriiformes', 'Family': 'Alcidae', 'CommonName': 'Marbled Murrelet', "Icun": 4}, # present in dataset
    {'Species': 'Tailed Frog', 'Order': 'Anura', 'Family': 'Ascaphidae', 'CommonName': 'Tailed Frog', "Icun": 1}, # present in dataset
    {'Species': 'Black Bear', 'Order': 'Carnivora', 'Family': 'Ursidae', 'CommonName': 'Black Bear', "Icun": 1}, # present in dataset
    {'Species': 'Cougar', 'Order': 'Carnivora', 'Family': 'Felidae', 'CommonName': 'Cougar', "Icun": 1},  # present in dataset
]

df_species = pd.DataFrame(data)
df_species

,Species,Order,Family,CommonName,Icun
0,Salmon,Salmoniformes,Salmonidae,Salmon,3
1,Seals,Carnivora,Phocidae,Seals,1
2,White Spotted Owl,Strigiformes,Strigidae,White Spotted Owl,3
3,Marbled Murrelet,Charadriiformes,Alcidae,Marbled Murrelet,4
4,Tailed Frog,Anura,Ascaphidae,Tailed Frog,1
5,Black Bear,Carnivora,Ursidae,Black Bear,1
6,Cougar,Carnivora,Felidae,Cougar,1


### 0.2.2 Species iNaturalist

In [ ]:
# Create `species` table with subset by selected order-family pairs. Runs in multiple threads.
# TODO: proper multithread with DuckDB
import concurrent.futures

dfs = []

# Define a function to execute the SQL query
def execute_sql(row):
    _df = con.sql(f"""
        SELECT
            EXTRACT(year FROM eventDate) AS year,
            EXTRACT(month FROM eventDate) AS month,
            EXTRACT(day FROM eventDate) AS day,
            decimalLatitude, decimalLongitude, taxonID,
            scientificName,
            taxonRank, kingdom, phylum, class, "order", family, genus,
            '{row["CommonName"]}' AS commonName,
            '{row["Icun"]}' AS icun,
        FROM read_parquet('s3://open-demo-datasets/climatebase/inaturalist/inaturalist_partitioned_order.parquet/*/*.parquet')
        WHERE  "order" = '{row["Order"]}' AND family = '{row["Family"]}'
        AND year > 2013
        """).df()
    return _df

# Create a ThreadPoolExecutor with a specified number of threads
with concurrent.futures.ThreadPoolExecutor(max_workers=7) as executor:
    # Iterate through the rows in parallel and execute the SQL queries
    for i, row in df_species.iterrows():
        print(row['Species'])
        future = executor.submit(execute_sql, row)
        dfs.append(future.result())
        print('complete.')

# Now 'dfs' contains the results of the SQL queries executed in parallel


Salmon


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

complete.
Seals


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

complete.
White Spotted Owl


...

complete.
Black Bear


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

complete.
Cougar


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

complete.


In [ ]:
df_species_subset = pd.concat(dfs)
df_species_subset
# df_species_subset.to_csv("df_species_subset.csv")

,year,month,day,decimalLatitude,decimalLongitude,taxonID,scientificName,taxonRank,kingdom,phylum,class,order,family,genus,commonName,icun
0,2023,6,2,53.212226,-1.672210,47516,Oncorhynchus mykiss,species,Animalia,Chordata,Actinopterygii,Salmoniformes,Salmonidae,Oncorhynchus,Salmon,3
1,2020,5,25,40.720285,-74.838455,47518,Salmo trutta,species,Animalia,Chordata,Actinopterygii,Salmoniformes,Salmonidae,Salmo,Salmon,3
2,2023,6,6,44.247278,-103.417248,47516,Oncorhynchus mykiss,species,Animalia,Chordata,Actinopterygii,Salmoniformes,Salmonidae,Oncorhynchus,Salmon,3
3,2021,3,12,45.205524,-121.080954,47516,Oncorhynchus mykiss,species,Animalia,Chordata,Actinopterygii,Salmoniformes,Salmonidae,Oncorhynchus,Salmon,3
4,2018,7,29,50.772086,15.431421,47518,Salmo trutta,species,Animalia,Chordata,Actinopterygii,Salmoniformes,Salmonidae,Salmo,Salmon,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62217,2020,5,23,40.496080,-3.717940,118552,Felis catus,species,Animalia,Chordata,Mammalia,Carnivora,Felidae,Felis,Cougar,1
62218,2020,5,24,NaN,NaN,118552,Felis catus,species,Animalia,Chordata,Mammalia,Carnivora,Felidae,Felis,Cougar,1
62219,2019,4,7,37.292074,-121.740622,632299,Lynx rufus fasciatus,subspecies,Animalia,Chordata,Mammalia,Carnivora,Felidae,Lynx,Cougar,1
62220,2020,1,29,-51.065364,-72.709257,559645,Puma concolor concolor,subspecies,Animalia,Chordata,Mammalia,Carnivora,Felidae,Puma,Cougar,1


### 0.2.3 Project location polygons


In [ ]:
# Load polygon data
polygons = {"type":"FeatureCollection","features":[
    {"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[-124.07834518293446,47.969370128062714],[-124.09275225056355,47.65769005067943],[-123.83519261526286,47.51708591400088],[-123.24457877616702,47.4882214377926],[-122.9490098210141,47.77240772945806],[-123.08268684731141,47.994311153655005],[-123.55314706542877,47.994311153655005],[-124.07834518293446,47.969370128062714]]]}},
    {"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[-124.23870968617123,47.13634158679939],[-123.57805254581301,47.11965163334313],[-123.669062299107,46.71353797477893],[-124.20421957141853,46.67945693479116],[-124.23870968617123,47.13634158679939]]]}},
    {"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[-123.58109247131506,49.417481995608924],[-123.31983498608535,48.98349255449175],[-122.6339887127427,48.98992658870232],[-122.62895308510316,49.49877130349736],[-123.6315373332137,49.52004368491608],[-123.58109247131506,49.417481995608924]]]}},
    {"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[-124.42298868182819,49.82045571003852],[-123.81180327099763,49.81954094134314],[-123.76986667171789,49.56224476193789],[-124.31599136446302,49.51942882858503],[-124.42491814123767,49.69151694820951],[-124.42298868182819,49.82045571003852]]]}},
    {"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[-125.52116810658971,50.23627272181358],[-125.52336748513979,49.943827211620665],[-125.04740802696911,49.938861896746296],[-124.90239848345094,50.09400466249809],[-124.93277837797962,50.31669058365233],[-125.3657969413377,50.31441929505926],[-125.52116810658971,50.23627272181358]]]}},
    {"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[-125.85448655268314,49.28458446736968],[-126.02038434554545,49.25998525115671],[-126.15408270544718,49.157649878453086],[-125.9045456811015,49.07449345762679],[-125.77559071051124,49.00643346076474],[-125.62524506891076,48.96566946638245],[-125.56790800114253,48.87962129286839],[-125.41021026366082,48.854764255030155],[-125.29715554957785,48.81978378596287],[-125.13443553666283,48.70391819030108],[-124.90267773645967,48.8925205450412],[-125.85448655268314,49.28458446736968]]]}},
    {"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[-124.80176592624821,48.37180351511935],[-124.64273030454387,48.29879093973981],[-124.5309065042571,48.38621006165711],[-124.5962518058437,48.418056856849944],[-124.72772866960103,48.42534267304404],[-124.80176592624821,48.37180351511935]]]}}
    ]}
gdf_prior = gpd.GeoDataFrame.from_features(polygons)
gdf_prior['name'] = ['Olympic National Park', 'Aberdeen Site', 'Vancouver', 'Spipiyous Area', 'Campbel River Area', 'Pacific Rim', 'Hobuk']
gdf_prior['site_class'] = ['Reference A', 'Project Area B', 'Counterfactual', 'Project Area D', 'Project Area E', 'Reference B', 'Project Area F']

gdf_prior

,geometry,name,site_class
0,"POLYGON ((-124.07835 47.96937, -124.09275 47.6...",Olympic National Park,Reference A
1,"POLYGON ((-124.23871 47.13634, -123.57805 47.1...",Aberdeen Site,Project Area B
2,"POLYGON ((-123.58109 49.41748, -123.31983 48.9...",Vancouver,Counterfactual
3,"POLYGON ((-124.42299 49.82046, -123.81180 49.8...",Spipiyous Area,Project Area D
4,"POLYGON ((-125.52117 50.23627, -125.52337 49.9...",Campbel River Area,Project Area E
5,"POLYGON ((-125.85449 49.28458, -126.02038 49.2...",Pacific Rim,Reference B
6,"POLYGON ((-124.80177 48.37180, -124.64273 48.2...",Hobuk,Project Area F


In [ ]:
# Calculate area in Hectares (m^2 -> Ha)
gdf = gdf_prior
gdf.crs = 'EPSG:4326'
gdf = gdf.to_crs(epsg=3857) # temporary transform, to calculate
gdf['area_hectares'] = gdf['geometry'].area * 0.0001
gdf = gdf.to_crs('EPSG:4326')

# Data type transform. Facilitates DuckDB.
gdf['geometry'] = gdf.geometry.apply(shapely.force_2d).apply(str)

gdf.head()
gdf.to_csv('polygons.csv')

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1537: UserWarning:

Geometry column does not contain geometry.



In [ ]:
# Polygons to duckdb
con.sql("CREATE OR REPLACE TABLE project_sites AS SELECT *, ST_GeomFromText(geometry) as geometry_geom, ST_Area(geometry_geom) AS area FROM gdf")
con.sql("FROM project_sites")

┌──────────────────────┬──────────────────────┬────────────────┬───┬──────────────────────┬──────────────────────┐
│       geometry       │         name         │   site_class   │ … │    geometry_geom     │         area         │
│       varchar        │       varchar        │    varchar     │   │       geometry       │        double        │
├──────────────────────┼──────────────────────┼────────────────┼───┼──────────────────────┼──────────────────────┤
│ POLYGON ((-124.078…  │ Olympic National P…  │ Reference A    │ … │ POLYGON ((-124.078…  │  0.47878707014116656 │
│ POLYGON ((-124.238…  │ Aberdeen Site        │ Project Area B │ … │ POLYGON ((-124.238…  │   0.2577507199918997 │
│ POLYGON ((-123.581…  │ Vancouver            │ Counterfactual │ … │ POLYGON ((-123.581…  │   0.4425831920052792 │
│ POLYGON ((-124.422…  │ Spipiyous Area       │ Project Area D │ … │ POLYGON ((-124.422…  │  0.17028681637339488 │
│ POLYGON ((-125.521…  │ Campbel River Area   │ Project Area E │ … │ POLYGON ((-

### 0.2.4 JOIN project_sites & df_species_subset

In [ ]:
con.sql("""
CREATE OR REPLACE TABLE species_by_project_site AS
  SELECT df_species_subset.*, project_sites.*
  FROM df_species_subset
  RIGHT JOIN project_sites
  ON ST_Within(ST_Point(df_species_subset.decimalLongitude, df_species_subset.decimalLatitude), project_sites.geometry_geom);
""")

con.sql("SELECT * FROM species_by_project_site").df().head(2)

,year,month,day,decimalLatitude,decimalLongitude,taxonID,scientificName,taxonRank,kingdom,phylum,...,family,genus,commonName,icun,geometry,name,site_class,area_hectares,geometry_geom,area
0,2023,2,22,49.239765,-122.899617,41976,Lynx rufus,species,Animalia,Chordata,...,Felidae,Lynx,Cougar,1,POLYGON ((-123.58109247131506 49.4174819956089...,Vancouver,Counterfactual,840294.673828,"[0, 2, 112, 0, 55, 32, 52, 55, 2, 0, 0, 0, 1, ...",0.442583
1,2023,2,22,49.239510,-122.899422,41976,Lynx rufus,species,Animalia,Chordata,...,Felidae,Lynx,Cougar,1,POLYGON ((-123.58109247131506 49.4174819956089...,Vancouver,Counterfactual,840294.673828,"[0, 2, 112, 0, 55, 32, 52, 55, 2, 0, 0, 0, 1, ...",0.442583


In [ ]:
gdf_species_site = gpd.GeoDataFrame(con.sql("SELECT * FROM species_by_project_site").df())
gdf_species_site['geometry'] = gpd.GeoSeries.from_wkt(gdf_species_site['geometry'])
gdf_species_site.set_geometry("geometry")
gdf_species_site.head(2)

,year,month,day,decimalLatitude,decimalLongitude,taxonID,scientificName,taxonRank,kingdom,phylum,...,family,genus,commonName,icun,geometry,name,site_class,area_hectares,geometry_geom,area
0,2023,2,22,49.239765,-122.899617,41976,Lynx rufus,species,Animalia,Chordata,...,Felidae,Lynx,Cougar,1,"POLYGON ((-123.58109 49.41748, -123.31983 48.9...",Vancouver,Counterfactual,840294.673828,"[0, 2, 112, 0, 55, 32, 52, 55, 2, 0, 0, 0, 1, ...",0.442583
1,2023,2,22,49.239510,-122.899422,41976,Lynx rufus,species,Animalia,Chordata,...,Felidae,Lynx,Cougar,1,"POLYGON ((-123.58109 49.41748, -123.31983 48.9...",Vancouver,Counterfactual,840294.673828,"[0, 2, 112, 0, 55, 32, 52, 55, 2, 0, 0, 0, 1, ...",0.442583


In [ ]:
gdf_species_site[['decimalLatitude',	'decimalLongitude', 'commonName']].to_csv('final_species_site.csv')
gdf_prior[['name', 'geometry']].to_csv('final_sites.csv')

In [ ]:
import pydeck as pdk

# Define a layer to display on a map
species_layer = pdk.Layer(
    "HexagonLayer",
    gdf_species_site[['decimalLatitude',	'decimalLongitude', 'commonName']],
    get_position=["decimalLongitude", "decimalLatitude"],
    auto_highlight=True,
    elevation_scale=500,
    pickable=True,
    # elevation_range=[0, 3000],
    extruded=True,
    coverage=1,
)

# Use initial dataset for viz, because only `gdf_prior` geometry column renders, for unknown reason.
polygon_layer = pdk.Layer(
    "GeoJsonLayer",
    gdf_prior[['name', 'geometry']],
    opacity=0.2,
    stroked=True,
    filled=True,
    extruded=True,
    wireframe=True,
    get_fill_color=[200,255,0],
    get_line_color=[25, 55, 25],
)

# Set the viewport location
view_state = pdk.ViewState(
    latitude=47.9848959,
    longitude=-121.7017072,
    zoom=4,
    min_zoom=1,
    max_zoom=15,
    pitch=45,
    bearing=0,
)

# Render
r = pdk.Deck(layers=[polygon_layer, species_layer], initial_view_state=view_state)
r.to_html("hexagon_layer.html")

<IPython.core.display.Javascript object>

In [ ]:
df_grouped = con.sql("""
    SELECT count(*) AS count, commonName, name, "order", family, year, area_hectares, count / area_hectares AS count_per_hectare, geometry, Icun
    FROM species_by_project_site
    GROUP BY commonName, name, "order", family, year, area_hectares, geometry, Icun
""").df()
df_grouped.head()


,count,commonName,name,order,family,year,area_hectares,count_per_hectare,geometry,icun
0,10,Cougar,Vancouver,Carnivora,Felidae,2023,840294.673828,0.000012,POLYGON ((-123.58109247131506 49.4174819956089...,1
1,12,Cougar,Vancouver,Carnivora,Felidae,2022,840294.673828,0.000014,POLYGON ((-123.58109247131506 49.4174819956089...,1
2,12,Cougar,Vancouver,Carnivora,Felidae,2021,840294.673828,0.000014,POLYGON ((-123.58109247131506 49.4174819956089...,1
3,3,Cougar,Olympic National Park,Carnivora,Felidae,2020,882546.717122,0.000003,POLYGON ((-124.07834518293446 47.9693701280627...,1
4,1,Cougar,Aberdeen Site,Carnivora,Felidae,2021,467576.824109,0.000002,POLYGON ((-124.23870968617123 47.1363415867993...,1


In [ ]:
# Create the grouped bar plot
import plotly.express as px


# Create a grouped bar chart
fig = px.bar(df_grouped, x='year', y='count', color='name',
             facet_col='commonName', facet_col_wrap=3, barmode='group')

# Customize the appearance
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Count',
)


# Show the plot
fig.show()

In [ ]:
# Create the grouped bar plot
import plotly.express as px



df_grouped_sub = df_grouped.loc[df_grouped['name'].isin(['Pacific Rim', 'Hobuk'])]

# Create a grouped bar chart
fig = px.bar(df_grouped_sub, x='year', y='count', color='commonName',
             facet_col='name', barmode='group')

# Customize the appearance
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Count',
)





# Show the plot
fig.show()

### 0.2.5 Calculate all observations per project area (for normalization)

We calculate all observations corresponding to the set of `order` studied. This is mainly to enable agile querying - otherwise working with the entire table is untenable.

In [ ]:
import duckdb
from threading import Thread, current_thread
import random

# Create table to insert into
con.sql(f"""
CREATE OR REPLACE TABLE all_observations AS
  SELECT
    EXTRACT(year FROM eventDate) AS year,
    EXTRACT(month FROM eventDate) AS month,
    EXTRACT(day FROM eventDate) AS day,
    decimalLatitude,
    decimalLongitude,
    "order",
    family,
    'a' AS commonName,
    1 AS icun
  FROM read_parquet('s3://open-demo-datasets/climatebase/inaturalist/inaturalist_partitioned_order.parquet/*/*.parquet')

LIMIT 1
""")

In [ ]:
# Threading
def write_from_thread(duckdb_con, row):
    local_con = duckdb_con.cursor()
    thread_name = str(current_thread().name)
    local_con.sql(f"""
        INSERT INTO all_observations (year, month, day, decimalLatitude, decimalLongitude, "order", family, commonName, icun)
        SELECT
          EXTRACT(year FROM eventDate) AS year,
          EXTRACT(month FROM eventDate) AS month,
          EXTRACT(day FROM eventDate) AS day,
          decimalLatitude,
          decimalLongitude,
          "order",
          family,
          '{row["CommonName"]}' AS commonName,
          '{row["Icun"]}' AS icun,
          FROM read_parquet('s3://open-demo-datasets/climatebase/inaturalist/inaturalist_partitioned_order.parquet/*/*.parquet')
        WHERE "order" = '{row["Order"]}'
        AND family = '{row["Family"]}'
        AND year > 2013
        AND decimalLongitude > -140.870173
        AND decimalLongitude <  -120.274210
        AND	decimalLatitude > 41.900551
        AND	decimalLatitude < 59.839700
    """)

threads = []

# Create multiple writer and reader threads (in the same process)
# Pass in the same connection as an argument
for i, row in df_species.iterrows():
  threads.append(Thread(target=write_from_thread,
                        args=(con, row),
                        name='write_thread_'+str(i)))


# Kick off all threads in parallel
for thread in threads:
  thread.start()

# Ensure all threads complete before printing final results
for thread in threads:
  thread.join()

In [ ]:
con.sql("DELETE FROM all_observations WHERE commonName = 'a'") # Delete initial dummy row
con.sql("FROM all_observations").df()

In [ ]:
# All observations, by polygon
con.sql("""
CREATE OR REPLACE TABLE all_observations_by_project_site AS
  SELECT all_observations.*, project_sites.*
  FROM all_observations
  RIGHT JOIN project_sites
  ON ST_Within(ST_Point(all_observations.decimalLongitude, all_observations.decimalLatitude), project_sites.geometry_geom);
""")

df_count_sites = con.sql("""
  SELECT
    name, count(*) AS count_obs_site FROM all_observations_by_project_site
  GROUP BY name
  ORDER BY count_obs_site
  """).df()

df_count_sites

,name,count_obs_site
0,Hobuk,67
1,Aberdeen Site,94
2,Spipiyous Area,101
3,Olympic National Park,182
4,Campbel River Area,290
5,Pacific Rim,756
6,Vancouver,2526


# 1. Workflow

## 1.1 Sample sites and their location

In [ ]:
gdf_project_sites = gdf.loc[~gdf['name'].isin(["Olympic National Park", "Vancouver"])]
gdf_project_sites

,geometry,name,site_class,area_hectares
1,POLYGON ((-124.23870968617123 47.1363415867993...,Aberdeen Site,Project Area B,467576.824109
3,POLYGON ((-124.42298868182819 49.8204557100385...,Spipiyous Area,Project Area D,326133.459679
4,POLYGON ((-125.5211681065897 50.23627272181358...,Campbel River Area,Project Area E,408631.239315
5,POLYGON ((-125.85448655268314 49.2845844673696...,Pacific Rim,Reference B,481458.970356
6,POLYGON ((-124.80176592624821 48.3718035151193...,Hobuk,Project Area F,36571.040920


## 1.5 Biodiversity Conservation Value



In [ ]:
df_grouped = pd.merge(df_grouped, df_count_sites[['name', 'count_obs_site']], on='name', how='left')
df_grouped

,count,commonName,name,order,family,year,area_hectares,count_per_hectare,geometry,icun,count_obs_site
0,10,Cougar,Vancouver,Carnivora,Felidae,2023,840294.673828,0.000012,POLYGON ((-123.58109247131506 49.4174819956089...,1,2526
1,12,Cougar,Vancouver,Carnivora,Felidae,2022,840294.673828,0.000014,POLYGON ((-123.58109247131506 49.4174819956089...,1,2526
2,12,Cougar,Vancouver,Carnivora,Felidae,2021,840294.673828,0.000014,POLYGON ((-123.58109247131506 49.4174819956089...,1,2526
3,3,Cougar,Olympic National Park,Carnivora,Felidae,2020,882546.717122,0.000003,POLYGON ((-124.07834518293446 47.9693701280627...,1,182
4,1,Cougar,Aberdeen Site,Carnivora,Felidae,2021,467576.824109,0.000002,POLYGON ((-124.23870968617123 47.1363415867993...,1,94
...,...,...,...,...,...,...,...,...,...,...,...
254,26,Salmon,Campbel River Area,Salmoniformes,Salmonidae,2022,408631.239315,0.000064,POLYGON ((-125.5211681065897 50.23627272181358...,3,290
255,2,Seals,Aberdeen Site,Carnivora,Phocidae,2017,467576.824109,0.000004,POLYGON ((-124.23870968617123 47.1363415867993...,1,94
256,1,Seals,Hobuk,Carnivora,Phocidae,2014,36571.040920,0.000027,POLYGON ((-124.80176592624821 48.3718035151193...,1,67
257,3,Seals,Aberdeen Site,Carnivora,Phocidae,2014,467576.824109,0.000006,POLYGON ((-124.23870968617123 47.1363415867993...,1,94


In [ ]:
# Show max total observations
con.sql("""
    SELECT name, commonName, SUM(count) AS sum_all_years FROM df_grouped
    WHERE name != 'Vancouver'
    GROUP BY name, commonName
    ORDER BY sum_all_years DESC
""").df().head(5)

# Based on this table, Pacific Rim is reference

,name,commonName,sum_all_years
0,Pacific Rim,Black Bear,269.0
1,Pacific Rim,Marbled Murrelet,202.0
2,Pacific Rim,Seals,162.0
3,Olympic National Park,Black Bear,96.0
4,Pacific Rim,Salmon,89.0


In [ ]:
df_reference_site = con.sql("""
  SELECT name, commonName, SUM(count) AS sum_all_years_reference_site FROM df_grouped
  WHERE name = 'Pacific Rim'
  GROUP BY name, commonName
  """).df()
df_reference_site

,name,commonName,sum_all_years_reference_site
0,Pacific Rim,White Spotted Owl,29.0
1,Pacific Rim,Marbled Murrelet,202.0
2,Pacific Rim,Black Bear,269.0
3,Pacific Rim,Cougar,5.0
4,Pacific Rim,Salmon,89.0
5,Pacific Rim,Seals,162.0


In [ ]:
# Introduce "reference site" values column, to help with sql
df_grouped_2 = pd.merge(df_grouped, df_reference_site[['commonName', 'sum_all_years_reference_site']], on='commonName', how='left')

## 6. Relative Abundance Score

###Calculation
Quintile interval approach
#### Variables
- A = # species the identified number seen in a given year in the project site
- B = # species the identified number seen in a given year in the reference site
- C = B / 5

### Formula
- Rank 1 = 1 to C
- Rank 2 = C to 2C
- Rank 3 = 2C to 3C
- Rank 4 = 3C to 4C
- Rank 5 = 4C or greater


In [ ]:
# Calculate relavite abundance score, then normalize.
df_relative_abundance = con.sql("SELECT *, CEIL(count / sum_all_years_reference_site / 5)  * CAST(icun AS integer) AS relative_abundance FROM df_grouped_2").df()


In [ ]:
# Calculate total biovalue for each site. Sums biovalue of all species for each site.
df_bio_value = con.sql("""
  SELECT
    name,
    ROUND(SUM(relative_abundance), 2) AS site_biodiversity_value
    FROM df_relative_abundance
  WHERE name != 'Vancouver' AND name != 'Pacific Rim'
  GROUP BY name
  ORDER BY site_biodiversity_value DESC
""").df()
df_bio_value

,name,site_biodiversity_value
0,Campbel River Area,83.0
1,Aberdeen Site,70.0
2,Spipiyous Area,70.0
3,Olympic National Park,55.0
4,Hobuk,47.0


# Appendix I: Additional plots

In [ ]:
df_relative_abundance_sub[['year', 'relative_abundance', 'commonName']]

,year,relative_abundance,commonName
15,2016,3.0,White Spotted Owl
19,2020,3.0,White Spotted Owl
26,2019,3.0,White Spotted Owl
31,2021,3.0,White Spotted Owl
37,2023,3.0,White Spotted Owl
...,...,...,...
246,2015,1.0,Seals
249,2023,3.0,White Spotted Owl
250,2016,4.0,Marbled Murrelet
256,2014,1.0,Seals


In [ ]:
df_relative_abundance_sub = df_relative_abundance.loc[df_relative_abundance['name'].isin(['Hobuk'])]
df=df_relative_abundance_sub
#  Create a 2D histogram using Plotly Express
# fig = px.density_heatmap(df_relative_abundance_sub, x='year', y='commonName')
fig = px.density_heatmap(df, x='year', y='commonName',  nbinsx=11)


# Customize the appearance
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Relative Abundance',
    coloraxis_colorbar_title='Common Name',
)

# Show the plot
fig.show()